In [1]:
import torch
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
from deeprobust.graph.defense import GCN
from deeprobust.graph.global_attack import MetaApprox, Metattack
from deeprobust.graph.utils import *
from deeprobust.graph.data import Dataset
import argparse

from config import *
from utils import *
from metrics import *
###
import tensorflow as tf
import time
from models import GCN_dropedge


 # Settings
len_tmp = labels.shape[0]



import os
os.environ["CUDA_VISIBLE_DEVICES"]="4"

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = 'cpu'
np.random.seed(args.seed)
torch.manual_seed(args.seed)
if device != 'cpu':
    torch.cuda.manual_seed(args.seed)

data = Dataset(root='/tmp/', name=args.dataset, setting='nettack')
adj, features, labels = data.adj, data.features, data.labels
print(features.shape)
idx_train, idx_val, idx_test = data.idx_train, data.idx_val, data.idx_test
idx_unlabeled = np.union1d(idx_val, idx_test)

ptb_rate = 0.01
perturbations = int(ptb_rate * (adj.sum()//2))
adj, features, labels = preprocess(adj, features, labels, preprocess_adj=False)
print(features.shape)

def test(adj):
    ''' test on GCN '''

    # adj = normalize_adj_tensor(adj)
    gcn = GCN(nfeat=features.shape[1], nclass=labels.max().item()+1, nhid=256,
        dropout=0.0, with_relu=True, with_bias=False, weight_decay=0, device=device)
    gcn = gcn.to(device)
    gcn.fit(features, adj, labels, idx_train) # train without model picking
    # gcn.fit(features, adj, labels, idx_train, idx_val) # train with validation model picking
    output = gcn.output.cpu()
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))

    return acc_test.item()

KeyboardInterrupt: 

In [ ]:
train_mask = np.zeros(len_tmp)
train_mask[idx_train] = 1

val_mask = np.zeros(len_tmp)
val_mask[idx_val] = 1

test_mask = np.zeros(len_tmp)
test_mask[idx_test] = 1

optimizer = tf.keras.optimizers.Adam(learning_rate=args.lr)

#tuple_adj = sparse_to_tuple(adj.tocoo())
features_tensor = tf.convert_to_tensor(features,dtype=dtype)
#adj_tensor = tf.SparseTensor(*tuple_adj)

labels_tmp = F.one_hot(labels)


y_train_tensor = tf.convert_to_tensor(labels_tmp,dtype=dtype)
train_mask_tensor = tf.convert_to_tensor(train_mask)
y_test_tensor = tf.convert_to_tensor(labels_tmp,dtype=dtype)
test_mask_tensor = tf.convert_to_tensor(test_mask)
y_val_tensor = tf.convert_to_tensor(labels_tmp,dtype=dtype)
val_mask_tensor = tf.convert_to_tensor(val_mask)

best_test_acc = 0
best_val_acc_trail = 0
best_val_loss = 10000

# Setup Surrogate model

surrogate = GCN(nfeat=features.shape[1], nclass=labels.max().item()+1,weight_decay=0.0,
                                nhid=256, dropout=0, with_relu=True, with_bias=False, device='cpu').to('cpu')
surrogate.fit(features, adj, labels, idx_train, idx_val, patience=100)
# Setup Attack Model
model = Metattack(surrogate, nnodes=adj.shape[0], feature_shape=features.shape,
        attack_structure=True, attack_features=False, device='cpu', lambda_=0).to('cpu')
# Attack

model.attack(features, adj, labels, idx_train, idx_unlabeled, n_perturbations=perturbations, ll_constraint=False)
modified_adj = model.modified_adj




Perturbing graph:   0%|          | 0/50 [00:00<?, ?it/s]

GCN loss on unlabled data: 0.49987319111824036
GCN acc on unlabled data: 0.8569512740277156
attack loss: 0.2617972493171692


Perturbing graph:   2%|▏         | 1/50 [00:09<07:39,  9.39s/it]

GCN loss on unlabled data: 0.4988565742969513
GCN acc on unlabled data: 0.8582923558337059
attack loss: 0.2586739659309387


Perturbing graph:   4%|▍         | 2/50 [00:18<07:22,  9.21s/it]

...

Perturbing graph:  68%|██████▊   | 34/50 [17:53<10:49, 40.59s/it]

GCN loss on unlabled data: 0.5903906226158142
GCN acc on unlabled data: 0.8399642378185069
attack loss: 0.37824878096580505


Perturbing graph:  70%|███████   | 35/50 [18:31<09:59, 39.94s/it]

: 

: 

In [ ]:
modified_adj=sp.csr_array(modified_adj.int())

tuple_adj = sparse_to_tuple(modified_adj.tocoo())
adj_tensor = tf.SparseTensor(*tuple_adj)


7


In [ ]:

# Settings
optimizer = tf.keras.optimizers.Adam(learning_rate=args.lr)
model = GCN_dropedge(input_dim=features.shape[1], output_dim=labels.max().item()+1, adj=adj_tensor)


best_test_acc = 0
best_val_acc = 0
best_val_loss = 10000


curr_step = 0
for epoch in range(args.epochs):

    with tf.GradientTape() as tape:
        output = model.call((features_tensor),training=True)
        cross_loss = masked_softmax_cross_entropy(output, y_train_tensor,train_mask_tensor)
        lossL2 = tf.add_n([tf.nn.l2_loss(v) for v in model.trainable_variables])
        loss = cross_loss #+ args.weight_decay*lossL2
        grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    output = model.call((features_tensor), training=False)
    train_acc = masked_accuracy(output, y_train_tensor,train_mask_tensor)
    val_acc  = masked_accuracy(output, y_val_tensor,val_mask_tensor)
    val_loss = masked_softmax_cross_entropy(output, y_val_tensor, val_mask_tensor)
    test_acc  = masked_accuracy(output, y_test_tensor,test_mask_tensor)

    if val_acc > best_val_acc:
        curr_step = 0
        best_test_acc = test_acc
        best_val_acc = val_acc
        best_val_loss= val_loss
        # Print results

    else:
        curr_step +=1
    if curr_step > args.early_stop:
        print("Early stopping...")
        break

    print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(cross_loss),"val_loss=", "{:.5f}".format(val_loss),
    "train_acc=", "{:.5f}".format(val_acc), "val_acc=", "{:.5f}".format(val_acc),
    "test_acc=", "{:.5f}".format(best_test_acc))

0.0
Epoch: 0001 train_loss= 1.93524 val_loss= 1.89642 train_acc= 0.41365 val_acc= 0.41365 test_acc= 0.38984
Epoch: 0002 train_loss= 1.87406 val_loss= 1.85383 train_acc= 0.49398 val_acc= 0.49398 test_acc= 0.47485
Epoch: 0003 train_loss= 1.81402 val_loss= 1.81144 train_acc= 0.51004 val_acc= 0.51004 test_acc= 0.49799
Epoch: 0004 train_loss= 1.75464 val_loss= 1.76905 train_acc= 0.50602 val_acc= 0.50602 test_acc= 0.49799
Epoch: 0005 train_loss= 1.69535 val_loss= 1.72631 train_acc= 0.50602 val_acc= 0.50602 test_acc= 0.49799
Epoch: 0006 train_loss= 1.63565 val_loss= 1.68287 train_acc= 0.50201 val_acc= 0.50201 test_acc= 0.49799
Epoch: 0007 train_loss= 1.57523 val_loss= 1.63880 train_acc= 0.51004 val_acc= 0.51004 test_acc= 0.49799
Epoch: 0008 train_loss= 1.51408 val_loss= 1.59434 train_acc= 0.52610 val_acc= 0.52610 test_acc= 0.56891
Epoch: 0009 train_loss= 1.45231 val_loss= 1.54956 train_acc= 0.55020 val_acc= 0.55020 test_acc= 0.58400
Epoch: 0010 train_loss= 1.39007 val_loss= 1.50448 train_acc=